# Application chaining

Use a notebook to demonstrate the application chaining:

- stage-in a pairs of EO products, pre- and post-event
- process three vegetation indexes: NDVI, NDWI and NBR
- process the burned area delineation products: NDVI/NDWI thresholds and RBR

### Chaining strategy

The application chaining prepares the inputs and parameters for executing the three CWL scripts and parses the outputs for the next step in the chaining.

The Jupyter Notebook is thus used to orchestrate the `cwltool` invoking the steps.

Applications to be chained in this scenario are a combination of Jupyter notebook based applications and Application Packages expressed as CWL that read a STAC local catalog and produce a local STAC catalog.

The applications are executed in Docker containers. These docker containers were built a tool called `repo2cli` that takes a remote git repository, creates the conda environment and create a CLI utility to execute the notebook using the Jupyter Notebook APIs (nbconvert).

This tool is available at: https://github.com/terradue-ogc-tb16/repo2cli and its docker at https://github.com/terradue-ogc-tb16/docker-repo2cli

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import yaml
from pystac import *
from helpers import * 

### First step: stage-in Sentinel-2 Level-2A acquisitions

Stage-in the pre- and post event acquisitions with the `stage-in` docker that takes as input a catalog entry reference

And produces as output:

- a local STAC catalog with a single collection and items whose assets point to the local staged bands (e.g. 'red', 'nir', 'swir16', 'swir22')

In [3]:
cwl_stage_in = 'stage-in.cwl'

In [4]:
inputs = get_workflow_inputs(cwl_stage_in)

inputs

{'input_reference': {'doc': 'A reference to an opensearch catalog',
  'label': 'A reference to an opensearch catalog',
  'type': 'string[]'},
 'target_folder': {'label': 'Folder to stage-in',
  'doc': 'Folder to stage-in',
  'type': 'string'}}

In [5]:
sentinel2_references = ['https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808',
                        'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348']

In [11]:
stagein_catalogs = []

for index, reference in enumerate(sentinel2_references):

    params = {'target_folder': './', 
              'input_reference': [reference]}
    
    result, std_err = process_worflow(cwl_stage_in, params)
    
    stagein_catalogs.append(get_catalog(result))
    
stagein_catalogs

/opt/anaconda/bin/cwltool --no-read-only --no-match-user stage-in.cwl#main /tmp/8l6kiffc
/opt/anaconda/bin/cwltool --no-read-only --no-match-user stage-in.cwl#main /tmp/rp6pj_f5


['file:///workspace/ogc-tb16/application-chaining/notebook/ss_5dhh_/catalog.json',
 'file:///workspace/ogc-tb16/application-chaining/notebook/6nzht0vd/catalog.json']

### Second step: vegetation index

The second step produces the vegetation indexes NDVI, NDWI and NBR. It takes as input the local STAC catalog.

This step produces as output:

- a local STAC catalog with a single item whose assets point to the 'NDVI', 'NDWI' and 'NBR' assets

The Jupyter Notebook git repository is available at https://github.com/terradue-ogc-tb16/vegetation-index

Its docker git repository is available at https://github.com/terradue-ogc-tb16/docker-vegetation-index

In [12]:
cwl_vegetation_index = 'vegetation-index.cwl'

In [13]:
inputs = get_workflow_inputs(cwl_vegetation_index)

inputs

{'aoi': {'doc': 'Area of interest in WKT',
  'label': 'Area of interest',
  'type': 'string'},
 'input_reference': {'doc': 'EO product for vegetation index',
  'label': 'EO product for vegetation index',
  'type': 'Directory[]'}}

In [14]:
aoi = 'POLYGON((136.508 -36.108,136.508 -35.654,137.178 -35.654,137.178 -36.108,136.508 -36.108))'

In [15]:
vegetation_index_catalogs = []

for index, stac_catalog in enumerate(stagein_catalogs):

    params = {'input_reference': [{'class': 'Directory', 'path': os.path.dirname(stac_catalog)}],
              'aoi': aoi}
    
    result, std_err = process_worflow(cwl_vegetation_index, params)
    
    vegetation_index_catalogs.append(get_catalog(result))
    
vegetation_index_catalogs    

/opt/anaconda/bin/cwltool --no-read-only --no-match-user vegetation-index.cwl#vegetation-index /tmp/w7jxpj18
/opt/anaconda/bin/cwltool --no-read-only --no-match-user vegetation-index.cwl#vegetation-index /tmp/cyflhrgb


['file:///workspace/ogc-tb16/application-chaining/notebook/sz1os_67/catalog.json',
 'file:///workspace/ogc-tb16/application-chaining/notebook/4wo1nqey/catalog.json']

### Third step - burned area delineation

This step takes as input a local STAC catalog with STAC catalog with a single item whose assets point to the 'NDVI', 'NDWI' and 'NBR' assets to produce a bitmask for the burned area based on the NDVI and NDWI thresholds and the Relativized Burn Ratio, an indicator of the burned area intensity .

The Jupyter Notebook git repository is available at https://github.com/terradue-ogc-tb16/burned-area-delineation

Its docker git repository is available at https://github.com/terradue-ogc-tb16/docker-burned-area-delineation

In [16]:
cwl_burned_area = 'burned-area-delineation.cwl'

In [17]:
inputs = get_workflow_inputs(cwl_burned_area)

inputs

{'ndvi_threshold': {'doc': 'NDVI difference threshold',
  'label': 'NDVI difference threshold',
  'type': 'string'},
 'ndwi_threshold': {'doc': 'NDWI difference threshold',
  'label': 'NDWI difference threshold',
  'type': 'string'},
 'post_event': {'doc': 'Post-event product for burned area delineation',
  'label': 'Post-event product for burned area delineation',
  'type': 'Directory'},
 'pre_event': {'doc': 'Pre-event product for burned area delineation',
  'label': 'Pre-event product for burned area delineation',
  'type': 'Directory'}}

Create the parameters file for CWL and invoke the CWL runner tool

In [18]:
ndvi_threshold = 0.19
ndwi_threshold = 0.18

In [19]:
params = {'ndvi_threshold': str(ndvi_threshold),
          'ndwi_threshold': str(ndwi_threshold),
          'pre_event': {'class': 'Directory', 'path': os.path.dirname(vegetation_index_catalogs[0])},
          'post_event': {'class': 'Directory', 'path': os.path.dirname(vegetation_index_catalogs[1])}
         }

result, std_err = process_worflow(cwl_burned_area, params)
 

/opt/anaconda/bin/cwltool --no-read-only --no-match-user burned-area-delineation.cwl#burned-area-delineation /tmp/hbh1je3c


### Inspect the results

In [20]:
cat = Catalog.from_file(get_catalog(result).replace('file://', ''))

In [21]:
cat.describe()

* <Catalog id=catalog>
  * <Item id=BURNED_AREA_DELINEATION>


In [22]:
item = next(cat.get_items())

item.assets

{'tvi': <Asset href=./BURNED_NDVI_NDWI_THRESHOLD.tif>,
 'dnbr': <Asset href=./BURNED_RBR.tif>}

In [23]:
print(item.assets['tvi'].get_absolute_href())
print(item.assets['dnbr'].get_absolute_href())

/workspace/ogc-tb16/application-chaining/notebook/s8adeufv/BURNED_AREA_DELINEATION/BURNED_NDVI_NDWI_THRESHOLD.tif
/workspace/ogc-tb16/application-chaining/notebook/s8adeufv/BURNED_AREA_DELINEATION/BURNED_RBR.tif
